First we need to load the sequence file

In [1]:
from datasets import load_dataset

trainFile="Data/s288c.fasta.train.txt"
testFile="Data/s288c.fasta.test.txt"
    

Then we will need to convert each sequence to codons, each codon is treated as a "word" and the sequence is the "sentence"

In [2]:
dataset=load_dataset('text', data_files=trainFile,split='train')



Using custom data configuration default-f555aa9781812fb7
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-f555aa9781812fb7/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


Now let's try to train a tokenlizer on our data

In [3]:
dataset

Dataset({
    features: ['text'],
    num_rows: 4990
})

In [4]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

In [5]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

This is where the tokenizer starts:

In [6]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

In [7]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=False)


In [8]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(),normalizers.StripAccents()]
)

In [9]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()


In [10]:
tokenizer.pre_tokenizer.pre_tokenize_str("CAA GTG ACC AGA ATG ATC ACC GGT GTT")


[('CAA', (0, 3)),
 ('GTG', (4, 7)),
 ('ACC', (8, 11)),
 ('AGA', (12, 15)),
 ('ATG', (16, 19)),
 ('ATC', (20, 23)),
 ('ACC', (24, 27)),
 ('GGT', (28, 31)),
 ('GTT', (32, 35))]

In [11]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=5000, special_tokens=special_tokens)

In [12]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)


In [13]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [14]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [15]:
encoding = tokenizer.encode("CAA GTG ACC AGA ATG ATC ACC GGT GTT")


In [16]:
encoding.tokens

['[CLS]',
 'CAA',
 'GTG',
 'ACC',
 'AGA',
 'ATG',
 'ATC',
 'ACC',
 'GGT',
 'GTT',
 '[SEP]']

In [17]:
encoding.type_ids


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [18]:
tokenizer.decoder = decoders.WordPiece(prefix="##")
from transformers import BertTokenizerFast
tokenizer.save("./yeast_token")
tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)


In [19]:
from transformers import BertTokenizer
from transformers import AutoTokenizer




In [20]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [21]:
datasets = load_dataset("text", data_files={"train": trainFile, "validation": testFile})
print(datasets)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets)

Using custom data configuration default-858e7f4f9c294c37
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})
        

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

...

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})


In [22]:
import pandas as pd
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# block_size = tokenizer.model_max_length
block_size = 128

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

print(tokenized_datasets)
print(tokenized_datasets['train'][0])

Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-0ae28d13a783de51.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-66867cae49f3685c.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-af204d5064b13aba.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-4384e86bdc1c91ba.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-a3b3aceb23530972.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-5252c45a05459aef.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-69d23f44ca7d06f0.arrow


Loading cached processed dataset at /home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-eb79a1f49f155516.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4990
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})
{'input_ids': [2, 56, 27, 25, 37, 56, 81, 54, 45, 54, 41, 73, 45, 27, 25, 78, 25, 53, 25, 30, 78, 81, 33, 46, 78, 25, 81, 76, 78, 80, 41, 70, 25, 44, 40, 25, 34, 78, 31, 23, 69, 44, 53, 44, 48, 25, 54, 55, 52, 31, 58, 23, 54, 36, 78, 31, 40, 78, 40, 31, 86, 41, 30, 25, 78, 53, 35, 27, 45, 31, 33, 31, 54, 28, 36, 53, 44, 28, 34, 23, 67, 59, 66, 44, 79, 81, 57, 38, 67, 28, 56, 52, 25, 71, 79, 42, 36, 53, 51, 53, 42, 85, 40, 33, 69, 64, 40, 34, 69, 40, 65, 37, 40, 31, 40, 30, 38, 65, 67, 36, 25, 54, 34, 67, 40, 25, 33, 25, 61, 28, 34, 40, 81, 35, 27, 27, 61, 61, 23, 72, 63, 23, 31, 31, 25, 47, 52, 81, 40, 38, 23, 34, 33, 28, 57, 52, 65, 30, 49, 71, 23, 27, 27, 54, 30, 45, 50, 54, 25, 61, 33, 47, 41, 27, 37, 31, 61, 81, 45, 31, 25, 

In [23]:
lm_datasets =tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)



#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
lm_datasets["train"][1]



{'input_ids': [61,
  28,
  34,
  40,
  81,
  35,
  27,
  27,
  61,
  61,
  23,
  72,
  63,
  23,
  31,
  31,
  25,
  47,
  52,
  81,
  40,
  38,
  23,
  34,
  33,
  28,
  57,
  52,
  65,
  30,
  49,
  71,
  23,
  27,
  27,
  54,
  30,
  45,
  50,
  54,
  25,
  61,
  33,
  47,
  41,
  27,
  37,
  31,
  61,
  81,
  45,
  31,
  25,
  40,
  85,
  41,
  33,
  25,
  41,
  74,
  23,
  64,
  36,
  25,
  36,
  42,
  49,
  23,
  25,
  56,
  61,
  23,
  27,
  70,
  25,
  88,
  57,
  54,
  25,
  31,
  34,
  63,
  36,
  23,
  37,
  41,
  36,
  25,
  42,
  27,
  31,
  23,
  30,
  34,
  38,
  40,
  40,
  23,
  71,
  33,
  42,
  35,
  67,
  31,
  40,
  49,
  48,
  78,
  41,
  33,
  46,
  45,
  27,
  41,
  47,
  49,
  52,
  27,
  23,
  61,
  48,
  38,
  47,
  36,
  27,
  41,
  30,
  33],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [25]:
print(lm_datasets["train"])
print(lm_datasets["train"][0])


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 19207
})
{'input_ids': [2, 56, 27, 25, 37, 56, 81, 54, 45, 54, 41, 73, 45, 27, 25, 78, 25, 53, 25, 30, 78, 81, 33, 46, 78, 25, 81, 76, 78, 80, 41, 70, 25, 44, 40, 25, 34, 78, 31, 23, 69, 44, 53, 44, 48, 25, 54, 55, 52, 31, 58, 23, 54, 36, 78, 31, 40, 78, 40, 31, 86, 41, 30, 25, 78, 53, 35, 27, 45, 31, 33, 31, 54, 28, 36, 53, 44, 28, 34, 23, 67, 59, 66, 44, 79, 81, 57, 38, 67, 28, 56, 52, 25, 71, 79, 42, 36, 53, 51, 53, 42, 85, 40, 33, 69, 64, 40, 34, 69, 40, 65, 37, 40, 31, 40, 30, 38, 65, 67, 36, 25, 54, 34, 67, 40, 25, 33, 25], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [34]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"CUBERT",
    evaluation_strategy = "epoch",
    learning_rate=1e-3,
    weight_decay=0.01,
)

from transformers import AutoConfig, AutoModelForCausalLM
model_checkpoint = "gpt2"

config = AutoConfig.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_config(config)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/lu/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_u

In [36]:
import torch
print(torch.cuda.is_available())


***** Running training *****
  Num examples = 19207
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7203


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")